# Imports

In [1]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
import re

# Distance Functions

In [2]:
# manhattan
def calc_l1_dist(a, b, c, x, y, z):  
    return (abs(int(a)-int(x)) + abs(b-y) + abs(c-z))

In [3]:
# euclidian
def calc_l2_dist(x1, x2, x3, y1, y2, y3):
    return (np.sqrt(np.pow(x1-y1,2) + np.pow(x2-y2,2) + np.pow(x3-y3,2)))

In [4]:
# RMSE
def calc_rmse(x1, x2, x3, y1, y2, y3):
    return (np.sqrt((np.pow(x1-y1,2) + np.pow(x2-y2,2) + np.pow(x3-y3,2))/2))

# Mosaic Functions

In [5]:
def find_nearest_img(r, g, b, data, dist):
    least_dist = np.inf
    least_row = None
    
    for row in data.values:
        if dist == 'l1':
            dist = calc_l1_dist(r, g, b, row[1], row[2], row[3])
        elif dist == 'l2':
            dist = calc_l2_dist(r, g, b, row[1], row[2], row[3])
        elif dist == 'rmse':
            dist = calc_rmse(r, g, b, row[1], row[2], row[3])

        if dist < least_dist:
            least_dist = dist
            least_row = row
            
    file = least_row[0]
    lowest_rgb = [[least_row[1], least_row[2], least_row[3]]]
        
    # # remove dupes
    # data = data.drop(data.index[data['Filename'] == file].tolist())
    
    # return file, data, [[least_row[1], least_row[2], least_row[3]]], least_dist
    return file, lowest_rgb, least_dist

In [6]:
def construct_mosaic(pixel_batch, r, g, b, data, dist, duplicate):
    # create empty matrix of target image size
    mosaic = np.zeros((r.shape[0], r.shape[1], 3))
    
    target_points = np.array([[0,0,0]])
    dist_points = np.array([[0,0,0]])
    batch_num = 0
    batches = [batch_num]
    
    total_dist = 0
    
    # iterate through RGB and find the closest related image
    for x in range(0, r.shape[0], pixel_batch):
        for y in range(0, r.shape[1], pixel_batch):

            # need to handle padding issues later
            r_batch = r[x:x+pixel_batch, y:y+pixel_batch]
            g_batch = b[x:x+pixel_batch, y:y+pixel_batch]
            b_batch = g[x:x+pixel_batch, y:y+pixel_batch]
            
            # compute batch averages
            r_avg = np.mean(r_batch)
            g_avg = np.mean(g_batch)
            b_avg = np.mean(b_batch)
            
            # save the avg values for plotting later
            target_points = np.append(target_points, [[r_avg, g_avg, b_avg]], axis=0)
            
            # find the closest image
            # returns the filename
            img, dist_point, least_dist = find_nearest_img(r_avg, b_avg, g_avg, data, dist)
            
            # remove dupes
            if not duplicate:
                data = data.drop(data.index[data['Filename'] == img].tolist())
            
            # accumulate the distance
            total_dist += least_dist
            
            # save the dist values for plotting later
            dist_points = np.append(dist_points, dist_point, axis=0)
            
            # extract the filename
            img = Image.open('data/images/'+img)
            
            # resize the image to be the batch size
            img = img.resize((pixel_batch, pixel_batch))#, Image.ANTIALIAS)
            
            # Convert to opencv image
            img = np.array(img) 
            img = img[:, :, ::-1].copy() 
            
            # add the image to the mosaic
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 0] = img[:,:,2]
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 1] = img[:,:,1]
            mosaic[x:x+pixel_batch, y:y+pixel_batch, 2] = img[:,:,0]
            
            batches.append(batch_num)
            batch_num += 1
    
    mosaic = Image.fromarray(mosaic.astype(np.uint8))
    return mosaic, target_points, dist_points, batches, total_dist

In [7]:
#read in the target image and divide it into RGB channels
def extract_rgb(target):
    image = cv2.imread(target)
    b,g,r = cv2.split(image)
    
    return[r, g, b]

# Result Handlers

In [8]:
def extract_targets_predicted(target_points, dist_points, color_idx):
    targets = target_points[:,color_idx]
    predicted = dist_points[:,color_idx]
    
    return targets, predicted

In [9]:
def plot(figure_title_size, subplot_title_size, axis_title_size, bin_range, figure_title, x_axis, y_axis, points, colors, filename, subplot_names):
    fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))
    fig.suptitle(figure_title, fontsize=figure_title_size)
    
    for row in range(len(axes)):
        for col in range(len(axes[row])):
            axes[row][col].hist(x=points[row,col], bins=bin_range, color=colors[row]);
            axes[row][col].set_title(subplot_names[row,col], fontsize=subplot_title_size)
            axes[row][col].set_xlabel(x_axis, fontsize=axis_title_size)  
            axes[row][col].set_ylabel(y_axis, fontsize=axis_title_size)
            axes[row][col].set_xticks(np.arange(0, 255, 20.0));
    fig.tight_layout()
    plt.savefig(filename)
    return plt

In [10]:
def parse_results(filename, figure_title):
    r_targets, r_predicted = extract_targets_predicted(target_points, dist_points, 0)
    g_targets, g_predicted = extract_targets_predicted(target_points, dist_points, 1)
    b_targets, b_predicted = extract_targets_predicted(target_points, dist_points, 2)
    
    figure_title_size=20
    subplot_title_size = 16
    axis_title_size = 13
    bin_range = list(range(0, 255))
    x_axis = 'Pixel Intensity'
    y_axis = '# of Pixels'

    points = np.array([[r_targets, r_predicted],
                       [g_targets, g_predicted],
                       [b_targets, b_predicted]])

    subplot_names = np.array([['Target R', 'Predicted R'],
                              ['Target G', 'Predicted G'],
                              ['Target B', 'Predicted B']])

    colors = ['darkred', 'darkgreen', 'darkblue']

    results = plot(figure_title_size, subplot_title_size, axis_title_size, bin_range, figure_title, x_axis, y_axis, points, colors, filename, subplot_names)
    results.show()

In [11]:
rgb_data = pd.read_csv('data/avg_database.csv')
rgb_data

,Filename,R Average,G Average,B Average
0,cifar_train_43545.jpeg,100.512695,103.308594,101.376953
1,cifar_train_26037.jpeg,160.252930,143.880859,98.750977
2,2333608.jpg,105.528021,117.531968,126.741077
3,cifar_test_8149.jpeg,83.702148,84.632812,63.346680
4,cifar_train_5736.jpeg,83.691406,92.091797,43.776367
...,...,...,...,...
94228,cifar_test_9995.jpeg,93.085938,123.327148,83.829102
94229,cifar_test_9996.jpeg,120.177734,132.110352,79.393555
94230,cifar_test_9997.jpeg,75.832031,106.240234,64.107422
94231,cifar_test_9998.jpeg,104.074219,95.823242,79.599609


# NO REPEATS

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'l1', duplicate=False)
# image.show()
image.save('data/target_norepeats_pb10_l1.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/L1_pb10_norepeats.png'
figure_title = 'Target and Predicted Pixel Values using L1 Distance'
parse_results(filename, figure_title)

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'l2', duplicate=False)
# image.show()
image.save('data/target_norepeats_pb10_l2.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/L2_pb10_norepeats.png'
figure_title = 'Target and Predicted Pixel Values using L2 Distance'
parse_results(filename, figure_title)

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'rmse', duplicate=False)
# image.show()
image.save('data/target_norepeats_pb10_rmse.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/rmse_pb10_norepeats.png'
figure_title = 'Target and Predicted Pixel Values using RMSE'
parse_results(filename, figure_title)

# REPEATS

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'l1', duplicate=True)
# image.show()
image.save('data/target_repeats_pb10_l1.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/L1_pb10_repeats.png'
figure_title = 'Target and Predicted Pixel Values using L1 Distance (Repeating Images)'
parse_results(filename, figure_title)

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'l2', duplicate=True)
# image.show()
image.save('data/target_repeats_pb10_l2.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/L2_pb10_repeats.png'
figure_title = 'Target and Predicted Pixel Values using L2 Distance (Repeating Images)'
parse_results(filename, figure_title)

In [ ]:
pixel_batch = 10
data = rgb_data.copy()
target = 'data/target.jpg'

r,g,b = extract_rgb(target)
image, target_points, dist_points, batches, total_dist = construct_mosaic(pixel_batch, r, g, b, data, 'rmse', duplicate=True)
# image.show()
image.save('data/target_repeats_pb10_rmse.jpg')
print('Total Distance of all points: ' + str(total_dist))

filename='data/figures/rmse_pb10_repeats.png'
figure_title = 'Target and Predicted Pixel Values using RMSE (Repeating Images)'
parse_results(filename, figure_title)

# Frame Compiler

In [ ]:
# pixel_batch = 10
# video_frames = 'data/videos/bunny_frames/frame'
# mosaic_frames = 'data/videos/mosaic_bunny_frames/frame'
# # frames = [1,2,4,5,6,7,10]
# # frames = [31,32,33,34,35,36,37,38,39,40]
# ext = '.jpg'
# total_frames = len(frames)#len(os.listdir(video_frames))
# regex = re.compile(r'\d+')

# # print('Total Frames: ', total_frames)

# # for frame in os.listdir(video_frames):
# # for frame in frames:
# for frame in range(201,251):
#     # recopy the saved rgb avg data because images are removed from the copy when used
#     data = rgb_data.copy()
    
#     # the path to the frame
#     file =  video_frames + str(frame) + ext
    
#     # extract RGB from the frame
#     r,g,b = extract_rgb(file)
    
#     # construct a mosaic for the frame
#     image = construct_mosaic(pixel_batch, r, g, b, data)
    
#     # retrieve the frame number from the file name and
#     # save the mosaic frame with the same frame number
#     # frame_num = regex.findall(frame)[0]
#     # image.save('data/videos/mosaic_bunny_frames/frame%d.jpg' % int(frame_num))
#     image.save(mosaic_frames + str(frame) + ext)
    
#     # inform the operator of the current progress
#     print('Processed frame {f}'.format(f=frame))

In [ ]:
# to test:
#     - threshold speed up
#     - use prev frame to determine if batch image should change
#     - padding
#     - RMSE and L1
#     - rather than using sub images, could you use parts of images to create the mosaic and get the original image?